# Downloding Applications from CTCAC or CDLAC

In [1]:
#install selenium
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [1]:
#import libraries and contingencies 
import openpyxl
import warnings
import multiprocess
from multiprocess import Pool, cpu_count
import PyPDF2
import pandas as pd
import re
import tabula
from pdfminer.high_level import extract_text
import numpy as np
import shutil
import os
import re
import PyPDF2
import requests
from io import BytesIO
import requests
from bs4 import BeautifulSoup
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
import requests

In [3]:
# Create a function to download PDFs
def dlpdf(url,file_path):
        # URL of the page containing the links
    url = url

    # Directory to save the downloaded PDFs
    download_directory = os.path.join(os.getcwd(), file_path)

    # Create a directory if it doesn't exist
    if not os.path.exists(download_directory):
        os.makedirs(download_directory)

    # Set up the Firefox WebDriver (Ensure GeckoDriver is in your system's PATH)
    driver = webdriver.Firefox()

    # Open the URL in the web browser
    driver.get(url)

    # Wait for a few seconds to allow JavaScript content to load (you may need to adjust this delay)
    driver.implicitly_wait(10)

    # Find all the anchor elements with href attributes containing ".pdf"
    pdf_links = driver.find_elements(By.CSS_SELECTOR, "a[href$='.pdf']")

    # Get the PDF URLs
    pdf_urls = [link.get_attribute("href") for link in pdf_links]

    # Download the PDFs
    for pdf_url in pdf_urls:
        response = requests.get(pdf_url)
        if response.status_code == 200:
            filename = os.path.join(download_directory, os.path.basename(pdf_url))
            with open(filename, "wb") as pdf_file:
                pdf_file.write(response.content)

    # Close the web browser
    driver.quit()
    

In [4]:
# Create a function to download Excel Files 

def justwannabringyoudownsobadly(url,file_path):
    url = url

    # Directory to save the downloaded PDFs
    download_directory = os.path.join(os.getcwd(), file_path)

    # Create a directory if it doesn't exist
    if not os.path.exists(download_directory):
        os.makedirs(download_directory)

    # Set up the Firefox WebDriver (Ensure GeckoDriver is in your system's PATH)
    driver = webdriver.Firefox()

    # Open the URL in the web browser
    driver.get(url)

    # Wait for a few seconds to allow JavaScript content to load (you may need to adjust this delay)
    driver.implicitly_wait(10)

    # Find all the anchor elements with href attributes containing ".pdf"
    pdf_links = driver.find_elements(By.CSS_SELECTOR, "a[href$='.xlsx']")

    # Get the PDF URLs
    pdf_urls = [link.get_attribute("href") for link in pdf_links]

    # Download the PDFs
    for pdf_url in pdf_urls:
        response = requests.get(pdf_url)
        if response.status_code == 200:
            filename = os.path.join(download_directory, os.path.basename(pdf_url))
            with open(filename, "wb") as pdf_file:
                pdf_file.write(response.content)


    # Close the web browser
    driver.quit()


# Create Executable

In [1]:
import tkinter as tk
from tkinter import filedialog
from functools import partial
import os
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

def download_files(url, file_type, file_path, browser):
    download_directory = os.path.join(os.getcwd(), file_path)
    if not os.path.exists(download_directory):
        os.makedirs(download_directory)

    # Debugging statement to print the value of url
    print("URL received:", url)

    # Validate the URL
    if not url:
        print("Error: URL cannot be empty.")
        return

    # Check if the URL starts with a valid scheme (e.g., http:// or https://)
    if not url.startswith(("http://", "https://")):
        url = "http://" + url

    try:
        if browser == 'Chrome':
            driver = webdriver.Chrome()
        elif browser == 'Firefox':
            driver = webdriver.Firefox()
        else:
            print("Error: Unsupported browser specified.")
            return

        driver.get(url)
        driver.implicitly_wait(10)

        if file_type == 'Excel':
            selector = "a[href$='.xlsx']"
        elif file_type == 'PDF':
            selector = "a[href$='.pdf']"
        else:
            driver.quit()
            return

        file_links = driver.find_elements(By.CSS_SELECTOR, selector)
        file_urls = [link.get_attribute("href") for link in file_links]

        for file_url in file_urls:
            response = requests.get(file_url)
            if response.status_code == 200:
                filename = os.path.join(download_directory, os.path.basename(file_url))
                with open(filename, "wb") as file:
                    file.write(response.content)

        driver.quit()

    except Exception as e:
        print("Error:", e)

def browse_button(entry):
    filename = filedialog.askdirectory()
    entry.delete(0, tk.END)
    entry.insert(0, filename)

def create_gui():
    window = tk.Tk()
    window.title("File Downloader")

    tk.Label(window, text="URL:").grid(row=0, column=0, sticky="w")
    url_entry = tk.Entry(window)
    url_entry.grid(row=0, column=1)

    tk.Label(window, text="File Type:").grid(row=1, column=0, sticky="w")
    file_type_var = tk.StringVar(window)
    file_type_var.set("Excel")  # default value
    file_type_menu = tk.OptionMenu(window, file_type_var, "Excel", "PDF")
    file_type_menu.grid(row=1, column=1)

    tk.Label(window, text="Browser:").grid(row=2, column=0, sticky="w")
    browser_var = tk.StringVar(window)
    browser_var.set("Firefox")  # default value
    browser_menu = tk.OptionMenu(window, browser_var, "Firefox", "Chrome")
    browser_menu.grid(row=2, column=1)

    tk.Label(window, text="Save Path:").grid(row=3, column=0, sticky="w")
    save_path_entry = tk.Entry(window)
    save_path_entry.grid(row=3, column=1)
    browse_button_func = partial(browse_button, save_path_entry)
    tk.Button(window, text="Browse", command=browse_button_func).grid(row=3, column=2)

    # Debugging statement to print the value of the URL entry widget
    def download_with_debug():
        print("URL entry value:", url_entry.get())
        download_files(url_entry.get(), file_type_var.get(), save_path_entry.get(), browser_var.get())
    tk.Button(window, text="Download", command=download_with_debug).grid(row=4, columnspan=2)

    window.mainloop()

if getattr(sys, 'frozen', False):  # When running as an executable
    # Set the current working directory to the script's directory
    os.chdir(sys._MEIPASS)

if __name__ == "__main__":
    create_gui()

In [7]:
pip install pyinstaller

Note: you may need to restart the kernel to use updated packages.


In [2]:
%run -m PyInstaller -F Download_Files.py

115727 INFO: PyInstaller: 6.4.0, contrib hooks: 2024.1
115728 INFO: Python: 3.9.16 (conda)
115728 INFO: Platform: Windows-10-10.0.22621-SP0
115729 INFO: wrote C:\Users\thech\Documents\GitHub\FairHousing\Download_Files.spec
115732 INFO: Extending PYTHONPATH with paths
['C:\\Users\\thech\\Documents\\GitHub\\FairHousing']
116092 INFO: checking Analysis
116108 INFO: Building because C:\Users\thech\Documents\GitHub\FairHousing\Download_Files.py changed
116109 INFO: Initializing module dependency graph...
116110 INFO: Caching module graph hooks...
116121 INFO: Analyzing base_library.zip ...
119035 INFO: Loading module hook 'hook-encodings.py' from 'C:\\Users\\thech\\anaconda3\\envs\\UDS\\lib\\site-packages\\PyInstaller\\hooks'...
120917 INFO: Loading module hook 'hook-pickle.py' from 'C:\\Users\\thech\\anaconda3\\envs\\UDS\\lib\\site-packages\\PyInstaller\\hooks'...
121409 INFO: Loading module hook 'hook-heapq.py' from 'C:\\Users\\thech\\anaconda3\\envs\\UDS\\lib\\site-packages\\PyInstaller\